In [1]:
import os
import re
import json
import random
import numpy as np
import pandas as pd
import datetime
from faker import Faker
from json_normalize import json_normalize

In [2]:
repo_path = os.getcwd()[:-11]

In [3]:
sales = pd.read_excel(f'{repo_path}data/sales_raw.xls', sheet_name = 'Orders')

sales.head()

,Row_ID,Order_Date,Customer_Name,City,Region,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Temperature,Is_Holiday,Fuel_Price
0,1,2016-11-08,Claire Gute,Henderson,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,42.31,False,2.572
1,2,2016-11-08,Claire Gute,Henderson,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,38.51,False,2.548
2,3,2016-06-12,Darrin Van Huff,Los Angeles,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,39.93,False,2.514
3,4,2015-10-11,Sean O'Donnell,Fort Lauderdale,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,46.63,True,2.561
4,5,2015-10-11,Sean O'Donnell,Fort Lauderdale,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,46.50,False,2.625


In [4]:
products = pd.read_csv(f'{repo_path}data/groceries_raw.csv')

products.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [5]:
sales.shape

(9994, 15)

In [6]:
products.shape

(38765, 3)

In [7]:
product_list = products.drop(
    columns = [
        'Member_number', 
        'Date'
    ]
).rename(
    columns = {
        'itemDescription': 'product'
    }
).drop_duplicates(
).reset_index(
    drop = True
)

In [8]:
product_list['product_id'] = product_list.reset_index()['index'].apply(
    lambda x: 'P' + f'{x + 1:04d}'
)

product_list['product'] = product_list['product'].apply(
    lambda x: x.title()
)

In [9]:
product_list.head()

,product,product_id
0,Tropical Fruit,P0001
1,Whole Milk,P0002
2,Pip Fruit,P0003
3,Other Vegetables,P0004
4,Rolls/Buns,P0005


In [10]:
len(product_list)

167

In [11]:
sales.drop(
    columns = [
        'Row_ID',
        'City',
        'Region',
        'Category',
        'Sub_Category',
        'Sales',
        'Discount',
        'Profit',
        'Temperature',
        'Is_Holiday',
        'Fuel_Price'
    ],
    inplace = True
)

sales.rename(
    columns = {
        'Order_Date': 'date_creation',
        'Customer_Name': 'name',
        'Product_Name': 'product_label',
        'Quantity': 'qty'
    },
    inplace = True
)

sales['name'] = sales['name'].apply(
    lambda x: x.replace('-', ' ')
)

In [12]:
sales.head()

,date_creation,name,product_label,qty
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3
2,2016-06-12,Darrin Van Huff,Self-Adhesive Address Labels for Typewriters b...,2
3,2015-10-11,Sean O'Donnell,Bretford CR4500 Series Slim Rectangular Table,5
4,2015-10-11,Sean O'Donnell,Eldon Fold 'N Roll Cart System,2


In [13]:
customers = sales[['name']].drop_duplicates().reset_index(drop = True)

customers['ref_customer'] = customers.reset_index()['index'].apply(
    lambda x: 'C' + f'{x + 1:04d}'
)

In [14]:
sales = sales.merge(
    customers,
    how = 'inner',
    left_on = ['name'],
    right_on = ['name']
)

In [15]:
invoices = sales[['date_creation', 'ref_customer']].drop_duplicates().reset_index(drop = True)

invoices['id'] = invoices.reset_index()['index'].apply(
    lambda x: 'IV' + f'{x + 1:05d}'
)

In [16]:
invoices.head()

,date_creation,ref_customer,id
0,2016-11-08,C0001,IV00001
1,2017-01-26,C0001,IV00002
2,2015-10-15,C0001,IV00003
3,2016-06-12,C0002,IV00004
4,2017-12-04,C0002,IV00005


In [17]:
sales = sales.merge(
    invoices,
    how = 'inner',
    left_on = ['date_creation', 'ref_customer'],
    right_on = ['date_creation', 'ref_customer']
)

In [18]:
sales

,date_creation,name,product_label,qty,ref_customer,id
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2,C0001,IV00001
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3,C0001,IV00001
2,2017-01-26,Claire Gute,"SimpliFile Personal File, Black Granite, 15w x...",2,C0001,IV00002
3,2015-10-15,Claire Gute,C-Line Cubicle Keepers Polyproplyene Holder w/...,6,C0001,IV00003
4,2015-10-15,Claire Gute,Xerox 1986,1,C0001,IV00003
...,...,...,...,...,...,...
9989,2016-05-03,Susan MacKendrick,Tennsco Industrial Shelving,5,C0793,IV04992
9990,2016-05-03,Susan MacKendrick,"Computer Room Manger, 14""",2,C0793,IV04992
9991,2016-05-03,Susan MacKendrick,Logitech VX Revolution Cordless Laser Mouse fo...,3,C0793,IV04992
9992,2016-05-03,Susan MacKendrick,Cisco 8961 IP Phone Charcoal,3,C0793,IV04992


In [19]:
sales['firstname'] = sales['name'].apply(
    lambda x: x.split(" ", 1)[0]
)

sales['lastname'] = sales['name'].apply(
    lambda x: x.split(" ", 1)[1] if len(x.split(" ", 1)) > 1 else np.nan
)

In [20]:
sales.head()

,date_creation,name,product_label,qty,ref_customer,id,firstname,lastname
0,2016-11-08,Claire Gute,Bush Somerset Collection Bookcase,2,C0001,IV00001,Claire,Gute
1,2016-11-08,Claire Gute,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",3,C0001,IV00001,Claire,Gute
2,2017-01-26,Claire Gute,"SimpliFile Personal File, Black Granite, 15w x...",2,C0001,IV00002,Claire,Gute
3,2015-10-15,Claire Gute,C-Line Cubicle Keepers Polyproplyene Holder w/...,6,C0001,IV00003,Claire,Gute
4,2015-10-15,Claire Gute,Xerox 1986,1,C0001,IV00003,Claire,Gute


In [21]:
all_products = product_list[['product']].values.tolist()

for i in range(len(all_products)):
    all_products[i] = all_products[i][0]

In [22]:
sale_products = sales[['product_label']].drop_duplicates().reset_index(drop = True)

sale_products['product_mapping'] = sale_products['product_label'].apply(
    lambda x: random.choice(all_products)
)

In [23]:
sale_products.head()

,product_label,product_mapping
0,Bush Somerset Collection Bookcase,Hair Spray
1,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Flower (Seeds)
2,"SimpliFile Personal File, Black Granite, 15w x...",Spread Cheese
3,C-Line Cubicle Keepers Polyproplyene Holder w/...,Nut Snack
4,Xerox 1986,Mustard


In [24]:
sales = sales.merge(
    sale_products,
    how = 'inner',
    left_on = ['product_label'],
    right_on = ['product_label']
).drop(
    columns = [
        'product_label'
    ]
).rename(
    columns = {
        'product_mapping': 'product_label'
    }
)

In [25]:
product_category = pd.read_csv(f'{repo_path}data/product_list_cat_map.csv')

product_category['type'] = product_category['type'].apply(
    lambda x: x.strip()
)

product_category.head()

,product,product_id,type
0,Tropical Fruit,P0001,Fruits&Vegetables
1,Whole Milk,P0002,Dairy
2,Pip Fruit,P0003,Fruits&Vegetables
3,Other Vegetables,P0004,Fruits&Vegetables
4,Rolls/Buns,P0005,Bakery


In [26]:
product_category.groupby('type')[['product']].count(
).reset_index(
).sort_values(
    by = 'product', 
    ascending = False
).reset_index(
    drop = True
)

,type,product
0,Dairy,20
1,Household items,20
2,Fruits&Vegetables,18
3,Meats,16
4,Snacks,15
5,Alcohol,12
6,Condiments,11
7,Dry/Baking Goods,11
8,Self-care,10
9,Beverages,9


In [27]:
category_price_range = pd.read_csv(f'{repo_path}data/category_price_range.csv')
category_price_range['max'] = category_price_range['max'].astype('float64')
category_price_range['min'] = category_price_range['min'].astype('str')
category_price_range['max'] = category_price_range['max'].astype('str')

category_price_range['range'] = category_price_range['min'] + ',' + category_price_range['max']

category_price_range.drop(
    columns = [
        'min',
        'max'
    ],
    inplace = True
)

In [28]:
product_category.head()

,product,product_id,type
0,Tropical Fruit,P0001,Fruits&Vegetables
1,Whole Milk,P0002,Dairy
2,Pip Fruit,P0003,Fruits&Vegetables
3,Other Vegetables,P0004,Fruits&Vegetables
4,Rolls/Buns,P0005,Bakery


In [29]:
product_category = product_category.merge(
    category_price_range,
    how = 'inner',
    left_on = ['type'],
    right_on = ['type']
)

In [30]:
product_category['subprice'] = product_category['range'].apply(
    lambda x: round(np.random.uniform(float(x.split(",", 1)[0]), float(x.split(",", 1)[1])), 2)
)

In [31]:
product_category.head()

,product,product_id,type,range,subprice
0,Tropical Fruit,P0001,Fruits&Vegetables,"2.0,6.0",3.77
1,Pip Fruit,P0003,Fruits&Vegetables,"2.0,6.0",4.90
2,Other Vegetables,P0004,Fruits&Vegetables,"2.0,6.0",2.55
3,Citrus Fruit,P0007,Fruits&Vegetables,"2.0,6.0",4.29
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,"2.0,6.0",5.80


In [32]:
sales = sales.merge(
    product_category,
    how = 'inner',
    left_on = ['product_label'],
    right_on = ['product']
).drop(
    columns = [
        'range'
    ]
).rename(
    columns = {
        'product': 'libelle'
    }
)

In [33]:
sales.head()

,date_creation,name,qty,ref_customer,id,firstname,lastname,product_label,libelle,product_id,type,subprice
0,2016-11-08,Claire Gute,2,C0001,IV00001,Claire,Gute,Hair Spray,Hair Spray,P0067,Self-care,4.55
1,2017-05-01,Arthur Gainer,3,C0152,IV01077,Arthur,Gainer,Hair Spray,Hair Spray,P0067,Self-care,4.55
2,2017-08-25,Seth Vernon,2,C0278,IV01976,Seth,Vernon,Hair Spray,Hair Spray,P0067,Self-care,4.55
3,2016-04-09,Herbert Flentye,5,C0521,IV03535,Herbert,Flentye,Hair Spray,Hair Spray,P0067,Self-care,4.55
4,2014-07-23,Alejandro Grove,2,C0009,IV00056,Alejandro,Grove,Hair Spray,Hair Spray,P0067,Self-care,4.55


In [34]:
sales.drop_duplicates(
    subset = ['id', 'product_id'],
    inplace = True
)

sales.reset_index(inplace = True, drop = True)

In [35]:
sales['price'] = sales['subprice'] * sales['qty']

total_paid = sales.groupby('id')[['price']].sum().reset_index().rename(columns = {'price': 'totalpaid'})

In [36]:
sales = sales.merge(
    total_paid,
    how = 'inner',
    left_on = ['id'],
    right_on = ['id']
).sort_values(by = ['date_creation', 'id'])

In [37]:
sales.head()

,date_creation,name,qty,ref_customer,id,firstname,lastname,product_label,libelle,product_id,type,subprice,price,totalpaid
8460,2014-01-03,Darren Powers,2,C0025,IV00183,Darren,Powers,Onions,Onions,P0066,Fruits&Vegetables,2.98,5.96,5.96
3871,2014-01-04,Phillina Ober,3,C0286,IV02019,Phillina,Ober,Cleaner,Cleaner,P0055,Household items,5.01,15.03,36.68
3872,2014-01-04,Phillina Ober,3,C0286,IV02019,Phillina,Ober,Beverages,Beverages,P0049,Beverages,2.27,6.81,36.68
3873,2014-01-04,Phillina Ober,2,C0286,IV02019,Phillina,Ober,Ham,Ham,P0029,Meats,7.42,14.84,36.68
651,2014-01-05,Mick Brown,3,C0470,IV03212,Mick,Brown,Nut Snack,Nut Snack,P0150,Snacks,6.42,19.26,19.26


In [38]:
product_category = product_category.drop(
    columns = [
        'range'
    ]
).rename(
    columns = {
        'product_id': 'id',
        'product': 'label',
        'subprice': 'price'
    }
)

In [39]:
synth = Faker()

product_category['description'] = product_category['label'].apply(
    lambda x: synth.text(max_nb_chars = 50)
)

In [40]:
product_category['cost_price'] = product_category['price'].apply(
    lambda x: round(np.random.uniform((x / 2.5), (x / 3.5)), 2)
)

In [41]:
product_category['date_creation'] = product_category['label'].apply(
    lambda x: synth.date_between(datetime.date(2012, 1, 1), datetime.date(2012, 12, 31))
)

product_category['date_modification'] = product_category['date_creation'].apply(
    lambda x: x + pd.Timedelta(days = np.random.randint(30, 180))
)

In [42]:
product_category.head()

,label,id,type,price,description,cost_price,date_creation,date_modification
0,Tropical Fruit,P0001,Fruits&Vegetables,3.77,Crime figure involve response reality.,1.40,2012-02-01,2012-03-17
1,Pip Fruit,P0003,Fruits&Vegetables,4.90,Value join pull energy class.,1.67,2012-10-28,2013-03-03
2,Other Vegetables,P0004,Fruits&Vegetables,2.55,Subject upon go. Follow lead different.,0.91,2012-10-18,2013-02-24
3,Citrus Fruit,P0007,Fruits&Vegetables,4.29,Open yes Mrs.,1.35,2012-09-03,2012-12-24
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,5.80,Structure history staff six bank face picture.,2.15,2012-03-17,2012-06-30


In [43]:
stock = product_category[['label', 'id', 'type', 'description', 'cost_price', 'date_creation']].copy()

In [44]:
perishable = [
    'Fruits&Vegetables',
    'Dairy',
    'Bakery',
    'Meats',
    'Instant Foods'
]

In [45]:
stock['qty'] = stock['type'].apply(
    lambda x: np.random.randint(50, 150) if x in perishable else np.random.randint(100, 300)
)

stock['expiry_date'] = stock.apply(
    lambda x: x.date_creation + pd.Timedelta(days = np.random.randint(5, 30)) if x.type in perishable else x.date_creation + pd.Timedelta(days = np.random.randint(30, 365)),
    axis = 1
)

stock['stock_id'] = stock.reset_index()['index'].apply(
    lambda x: 'S' + f'{x + 1:04d}'
)

stock['in_shelf'] = 'True'

In [46]:
stock.head()

,label,id,type,description,cost_price,date_creation,qty,expiry_date,stock_id,in_shelf
0,Tropical Fruit,P0001,Fruits&Vegetables,Crime figure involve response reality.,1.40,2012-02-01,70,2012-02-11,S0001,True
1,Pip Fruit,P0003,Fruits&Vegetables,Value join pull energy class.,1.67,2012-10-28,73,2012-11-12,S0002,True
2,Other Vegetables,P0004,Fruits&Vegetables,Subject upon go. Follow lead different.,0.91,2012-10-18,102,2012-11-10,S0003,True
3,Citrus Fruit,P0007,Fruits&Vegetables,Open yes Mrs.,1.35,2012-09-03,75,2012-09-30,S0004,True
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,Structure history staff six bank face picture.,2.15,2012-03-17,55,2012-03-30,S0005,True


In [47]:
product_category = product_category.merge(
    stock,
    left_on = ['id'],
    right_on = ['id']
)[['label_x', 'id', 'type_x', 'price', 'description_x', 'cost_price_x', 'date_creation_x', 'date_modification', 'stock_id']].rename(
    columns = {
        'label_x': 'label',
        'type_x': 'type',
        'description_x': 'description',
        'cost_price_x': 'cost_price',
        'date_creation_x': 'date_creation',
    }
)

In [48]:
product_category.head()

,label,id,type,price,description,cost_price,date_creation,date_modification,stock_id
0,Tropical Fruit,P0001,Fruits&Vegetables,3.77,Crime figure involve response reality.,1.40,2012-02-01,2012-03-17,S0001
1,Pip Fruit,P0003,Fruits&Vegetables,4.90,Value join pull energy class.,1.67,2012-10-28,2013-03-03,S0002
2,Other Vegetables,P0004,Fruits&Vegetables,2.55,Subject upon go. Follow lead different.,0.91,2012-10-18,2013-02-24,S0003
3,Citrus Fruit,P0007,Fruits&Vegetables,4.29,Open yes Mrs.,1.35,2012-09-03,2012-12-24,S0004
4,Packaged Fruit/Vegetables,P0013,Fruits&Vegetables,5.80,Structure history staff six bank face picture.,2.15,2012-03-17,2012-06-30,S0005


In [49]:
inventory = stock[['stock_id', 'date_creation', 'qty', 'type']].copy()

In [50]:
inventory2 = inventory.copy()

inventory2['date_creation'] = inventory2.date_creation.apply(
    lambda x: x + pd.Timedelta(days = 7)
)

inventory2['qty'] = inventory2.apply(
    lambda x: x.qty - np.random.randint(15, x.qty - 25) if x.type in perishable else  x.qty - np.random.randint(25, x.qty - 50),
    axis = 1
)

In [51]:
inventory3 = inventory2.copy()

inventory3['date_creation'] = inventory3.date_creation.apply(
    lambda x: x + pd.Timedelta(days = 7)
)

inventory3['qty'] = inventory3.apply(
    lambda x: x.qty - np.random.randint(5, x.qty - 5) if x.type in perishable else  x.qty - np.random.randint(20, x.qty - 15),
    axis = 1
)

In [52]:
inventory = pd.concat(
    [
        inventory, 
        inventory2, 
        inventory3
    ]
).drop(
    columns = [
        'type'
    ]
).reset_index(
    drop = True
).rename(
    columns = {
        'date_creation': 'date'
    }
).reset_index()

inventory['date'] = inventory['date'].astype('str')

In [53]:
inventory.head()

,index,stock_id,date,qty
0,0,S0001,2012-02-01,70
1,1,S0002,2012-10-28,73
2,2,S0003,2012-10-18,102
3,3,S0004,2012-09-03,75
4,4,S0005,2012-03-17,55


---
## Populate to JSON

#### 1. Invoices

In [54]:
with open(f'{repo_path}data/dolibarr_inv_sample.json') as data_file:    
    dolibarr_inv_sample = json.load(data_file)
    data_file.close()

In [55]:
inv_dict_sample = dolibarr_inv_sample[1]

In [56]:
for key in inv_dict_sample:
    if key == 'lines':
        for line_key in inv_dict_sample.get(key)[0]:
            inv_dict_sample.get(key)[0].update({line_key: None})
    else:
        inv_dict_sample.update({key: None})
        
del inv_dict_sample.get('lines')[1]

In [57]:
invoices = []
for i in range(len(sales.id.unique())):
    invoices.append(inv_dict_sample.copy())

In [58]:
inv_ids = sales.id.unique().tolist()

In [59]:
for i in range(len(invoices)):
    
    invoices[i].update({'id': inv_ids[i]})
    inv = sales.loc[sales.id == inv_ids[i]][['date_creation', 'name', 'ref_customer', 'firstname', 'lastname', 'totalpaid']].drop_duplicates().reset_index(drop = True)
    
    invoices[i].update({'date_creation': str(inv[['date_creation']].iloc[0][0])[:10]})
    invoices[i].update({'name': inv[['name']].iloc[0][0]})
    invoices[i].update({'ref_customer': inv[['ref_customer']].iloc[0][0]})
    invoices[i].update({'firstname': inv[['firstname']].iloc[0][0]})
    invoices[i].update({'lastname': inv[['lastname']].iloc[0][0]})
    invoices[i].update({'totalpaid': float(round(inv[['totalpaid']].iloc[0][0], 2))})
    
    inv_lines = sales.loc[sales.id == inv_ids[i]].reset_index(drop = True)
    prods = len(inv_lines)
    lines = []
    
    for p in range(prods):
        lines_dict = inv_dict_sample.get('lines')[0].copy()
        lines_dict.update({'ref': inv_lines[['product_id']].iloc[p][0]})
        lines_dict.update({'product_ref': inv_lines[['product_id']].iloc[p][0]})
        lines_dict.update({'fk_product': inv_lines[['product_id']].iloc[p][0]})
        lines_dict.update({'libelle': inv_lines[['libelle']].iloc[p][0]})
        lines_dict.update({'product_label': inv_lines[['libelle']].iloc[p][0]})
        lines_dict.update({'qty': int(inv_lines[['qty']].iloc[p][0])})
        lines_dict.update({'subprice': float(inv_lines[['subprice']].iloc[p][0])})
        lines_dict.update({'price': float(inv_lines[['price']].iloc[p][0])})
        lines_dict.update({'product_type': inv_lines[['type']].iloc[p][0]})
        lines_dict.update({'fk_product_type': inv_lines[['type']].iloc[p][0]})
        lines.append(lines_dict)
        
    invoices[i].update({'lines': lines})
    
    
    

#### 2. Products

In [60]:
with open(f'{repo_path}data/dolibarr_product_sample.json') as data_file:    
    dolibarr_prod_sample = json.load(data_file)
    data_file.close()

In [61]:
prod_dict_sample = dolibarr_prod_sample[1]

for key in prod_dict_sample:
    prod_dict_sample.update({key: None})

In [62]:
prods = []
for i in range(len(product_category)):
    prods.append(prod_dict_sample.copy())

In [63]:
prod_ids = product_category.id.unique().tolist()

In [64]:
for i in range(len(prods)):
    
    prods[i].update({'id': prod_ids[i]})
    prods[i].update({'ref': prod_ids[i]})
    
    prd = product_category.loc[product_category.id == prod_ids[i]].reset_index(drop = True)
    
    prods[i].update({'date_creation': str(prd[['date_creation']].iloc[0][0])[:10]})
    prods[i].update({'date_modification': str(prd[['date_modification']].iloc[0][0])[:10]})
    
    prods[i].update({'price': float(round(prd[['price']].iloc[0][0], 2))})
    prods[i].update({'cost_price': float(round(prd[['cost_price']].iloc[0][0], 2))})
    
    prods[i].update({'type': prd[['type']].iloc[0][0]})
    prods[i].update({'description': prd[['description']].iloc[0][0]})
    prods[i].update({'label': prd[['label']].iloc[0][0]})
    prods[i].update({'fk_unit': prd[['stock_id']].iloc[0][0]})

#### 3. Stocks

In [65]:
with open(f'{repo_path}data/dolibarr_stock_sample.json') as data_file:    
    dolibarr_stock_sample = json.load(data_file)
    data_file.close()

In [66]:
stock_dict_sample = dolibarr_stock_sample[1]

for key in stock_dict_sample:
    stock_dict_sample.update({key: None})

In [67]:
stocks = []
for i in range(len(stock)):
    stocks.append(stock_dict_sample.copy())

In [68]:
stock_ids = stock.stock_id.unique().tolist()

In [69]:
for i in range(len(stocks)):
    
    stocks[i].update({'id': stock_ids[i]})
    stocks[i].update({'ref': stock_ids[i]})
    
    stk = stock.loc[stock.stock_id == stock_ids[i]].reset_index(drop = True)
    
    stocks[i].update({'date_creation': str(stk[['date_creation']].iloc[0][0])[:10]})
    stocks[i].update({'eatby': str(stk[['expiry_date']].iloc[0][0])[:10]})
    
    stocks[i].update({'price': float(round(stk[['cost_price']].iloc[0][0], 2))})
    stocks[i].update({'qty': int(round(stk[['qty']].iloc[0][0], 2))})
    
    stocks[i].update({'type': stk[['type']].iloc[0][0]})
    stocks[i].update({'label': stk[['label']].iloc[0][0]})
    stocks[i].update({'product_id': stk[['id']].iloc[0][0]})
    stocks[i].update({'status': stk[['in_shelf']].iloc[0][0]})

#### 4. Inventory

In [70]:
dolibarr_inventory_sample = {
    'stock_id': None,
    'date': None,
    'qty': None,
}

In [71]:
inventories = []
for i in range(len(inventory)):
    inventories.append(dolibarr_inventory_sample.copy())

In [72]:
for i in range(len(inventory)):
    
    invt = inventory.loc[inventory['index'] == i]
    
    inventories[i].update({'qty': int(round(invt[['qty']].iloc[0][0], 2))})
    
    inventories[i].update({'stock_id': invt[['stock_id']].iloc[0][0]})
    inventories[i].update({'date': invt[['date']].iloc[0][0]})

### Output

In [73]:
# output json files

with open(f'{repo_path}data/dolibarr_invoices.json', 'w') as file:
    json.dump(invoices, file)
    file.close()
    
with open(f'{repo_path}data/dolibarr_products.json', 'w') as file:
    json.dump(prods, file)
    file.close()
    
with open(f'{repo_path}data/dolibarr_stocks.json', 'w') as file:
    json.dump(stocks, file)
    file.close()
    
with open(f'{repo_path}data/dolibarr_inventory.json', 'w') as file:
    json.dump(inventories, file)
    file.close()

## End.